## [Decolonise Science's Metadata extraction](https://github.com/AfricArxiv/decolonise-science/wiki/Metadata-extraction)

I adapted the `zenodo-community-stats.sh`  .. to work on Colab.

Just run all the cells below to the end to get your extracted CSV file 

In [1]:
! sudo apt-get install curl jq

Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.17).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'sudo apt autoremove' to remove them.
The following additional packages will be installed:
  libjq1 libonig4
The following NEW packages will be installed:
  jq libjq1 libonig4
0 upgraded, 3 newly installed, 0 to remove and 42 not upgraded.
Need to get 276 kB of archives.
After this operation, 930 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libonig4 amd64 6.7.0-1 [119 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libjq1 amd64 1.5+dfsg-2 [111 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 jq amd64 1.5+dfsg-2 [45.6 kB]
Fetched 276 kB in 0s (894 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable

We are using Python to set the variables because they do not get recognized just using `!`.

If you want to change the name of your extracted file, change the vairable `FILENAME`.

In [2]:
import os
from datetime import datetime

# current date and time
curDT = datetime.now()
# current date
date = curDT.strftime("%Y-%m-%d")
# creating the date object of today's date
os.environ["date"] =  date
# Change below for your filename
FILENAME = f"lanafric_zenodo-extract_{date}"
os.environ["filename"] = FILENAME

In [3]:
# To use this script you need to have "curl" and "jq" installed.

# Download all records (including multiple versions) from the community (max 10k records)
!curl "https://zenodo.org/api/records/?page=1&size=10000&q=&all_versions=true&communities=africarxiv" > zenodo-resp.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 96874  100 96874    0     0  45933      0  0:00:02  0:00:02 --:--:-- 45933


In [ ]:

# Create CSV file header
!echo "URL,DOI,Title,Abstract,Views,Downloads,Author,Affiliation,ORCID,Created,Updated,License,Keywords" > "${filename}.csv"
# Process response and append to CSV file
# Take first author information, parse timestamps, join keywords into a single string if array
!jq -r '.hits.hits[] | [.links.self, .metadata.doi, .metadata.title, .metadata.description, .stats.views, .stats.downloads, .metadata.creators[0].name, .metadata.creators[0].affiliation, .metadata.creators[0].orcid, (.created | split(".")[0] + "Z" | fromdate | strftime("%Y-%m-%d")), (.updated | split(".")[0] + "Z" | fromdate | strftime("%Y-%m-%d")), .metadata.license.id, (.metadata.keywords | if type == "array" then join(", ") else . end)] | @csv' zenodo-resp.json >> "${filename}.csv"

# Add Excel UTF-8 BOM to CSV file
!echo -ne "\xEF\xBB\xBF" | cat - "${filename}.csv" > temp.csv && mv temp.csv "${filename}.csv"


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  864k  100  864k    0     0   187k      0  0:00:04  0:00:04 --:--:--  211k
